# Проект e-commerce

 <p style="font-size:18px">Необходимо проанализировать совершенные пользователями покупки и ответить на вопросы:</p>
 <ul style="font-size: 20px; line-height: 1.5;">
  <li>1. Сколько пользователей, которые совершили покупку только один раз?</li>
  <li>2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?</li>
  <li>3. По каждому товару определить, в какой день недели товар чаще всего покупается.</li>
  <li>4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?</li>
  <li>5. Выполнить когортный анализ пользователей. В период с января по декабрь выявите когорту с самым высоким retention на 3-й месяц.</li>
  <li>6. Построить RFM-сегментацию пользователей.</li>
</ul>

##### Описание данных:
<li>olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей</li></br>

customer_id — позаказный идентификатор пользователя

customer_unique_id —  уникальный идентификатор пользователя

customer_zip_code_prefix —  почтовый индекс пользователя

customer_city —  город доставки пользователя

customer_state —  штат доставки пользователя

<li>olist_orders_dataset.csv —  таблица заказов</li></br>

order_id —  уникальный идентификатор заказа

customer_id —  позаказный идентификатор пользователя

order_status —  статус заказа

order_purchase_timestamp —  время создания заказа

order_approved_at —  время подтверждения оплаты заказа

order_delivered_carrier_date —  время передачи заказа в логистическую службу

order_delivered_customer_date —  время доставки заказа

order_estimated_delivery_date —  обещанная дата доставки

<li>olist_order_items_dataset.csv —  товарные позиции, входящие в заказы</li></br>

order_id —  уникальный идентификатор заказа

order_item_id —  идентификатор товара внутри одного заказа

product_id —  ид товара

seller_id — ид производителя товара

shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике

price —  цена за единицу товара

freight_value —  вес товара


##### Уникальные статусы заказов в таблице olist_orders_dataset:

created —  создан  
approved —  подтверждён  
invoiced —  выставлен счёт  
processing —  в процессе сборки заказа  
shipped —  отгружен со склада  
delivered —  доставлен пользователю  
unavailable —  недоступен  
canceled —  отменён

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Импортирую библиотеки
</div>

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Данные загружены локально
</div>


In [135]:
olist_customers_dataset = pd.read_csv('olist_customers_dataset.csv') # таблица с уникальными идентификаторами пользователей
olist_orders_dataset = pd.read_csv('olist_orders_dataset.csv') # таблица заказов
olist_order_items_dataset = pd.read_csv('olist_order_items_dataset.csv') # товарные позиции, входящие в заказы

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Объединяю датасеты
</div>

In [136]:
df_merged1 = pd.merge(olist_customers_dataset, olist_orders_dataset, on='customer_id', how='inner')

In [137]:
df_merged = pd.merge(df_merged1, olist_order_items_dataset, on='order_id', how='left')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Проверяю пропуски
</div>

In [138]:
df_merged.isnull().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
dtype: int64

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Проверяю дубликаты
</div>

In [139]:
df_merged.duplicated().sum()

np.int64(0)

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Проверяю типы данных
</div>

In [140]:
df_merged.dtypes

customer_id                       object
customer_unique_id                object
customer_zip_code_prefix           int64
customer_city                     object
customer_state                    object
order_id                          object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
order_item_id                    float64
product_id                        object
seller_id                         object
shipping_limit_date               object
price                            float64
freight_value                    float64
dtype: object

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Преобразую столбцы с датами в datetime формат
</div>

In [141]:
df_merged[
    [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
        'shipping_limit_date'
    ]
] = df_merged[
    [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
        'shipping_limit_date'
    ]
].apply(pd.to_datetime)


<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Проверяю временные рамки транзакций
</div>

In [142]:
print(f'min',df_merged.order_purchase_timestamp.min(), f'max', df_merged.order_purchase_timestamp.max())

min 2016-09-04 21:15:19 max 2018-10-17 17:30:18


<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Смотрю на распредление заказов в зависимости от статуса
</div>

In [143]:
df_merged.groupby("order_status", as_index = False).agg({"customer_id": "count"})

,order_status,customer_id
0,approved,3
1,canceled,706
2,created,5
3,delivered,110197
4,invoiced,361
5,processing,357
6,shipped,1186
7,unavailable,610


#### 1. Сколько пользователей, которые совершили покупку только один раз?

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
В качестве покупки буду рассматривать подтверждение заказа (order_approved_at)
</div>

In [144]:
task1 = df_merged[df_merged['order_approved_at'].notnull()]

In [145]:
filtered_df = task1.groupby('customer_unique_id').agg({'order_id': 'nunique'})

In [146]:
single_purchase_users = len(filtered_df[filtered_df['order_id'] == 1])
single_purchase_users

93049

#### 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Недоставленными считаю заказы со статусом "canceled" и "unavailable"
</div>

In [147]:
not_delivered_df = (
    df_merged[df_merged['order_status'].isin(['unavailable', 'canceled'])]
    .groupby('order_status')
    .agg({'order_id': 'nunique'})
    .rename(columns={'order_id': 'order_id_cnt'})
)
not_delivered_df

,order_id_cnt
order_status,
canceled,625
unavailable,609


<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
На этапе предварительного анализа я выяснил, что разница между самой ранней и самой поздней транзакцией составляет 2г 1месяц или 25 месяцев
</div>

In [148]:
not_delivered_df / 25

,order_id_cnt
order_status,
canceled,25.00
unavailable,24.36


#### 3. По каждому товару определить, в какой день недели товар чаще всего покупается.

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Дублирую дф из первого вопроса, т.к. в нем я дал определение покупке
</div>

In [149]:
task3 = task1.copy()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Добавляю столбец с днями недели
</div>

In [150]:
task3['day_of_week'] = task1['order_purchase_timestamp'].dt.day_name()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Считаю количество покупок товара по дням недели
</div>

In [151]:
product_day_counts = task2.groupby(['product_id', 'day_of_week']).size().reset_index(name='purchase_count')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Оставляю максимальное количество покупок у каждого товара
</div>

In [152]:
max_counts = product_day_counts.groupby('product_id')['purchase_count'].max().reset_index(name='max_purchase_count')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Объединяю максимальное количество покупок с исходной таблицей
</div>

In [153]:
result = product_day_counts.merge(max_counts, on='product_id')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Оставляю только те строки, которые равны максимальному значению, чтобы учесть случаи, когда товар продавался одинаково много в различные дни недели
</div>

In [154]:
result[result['purchase_count'] == result['max_purchase_count']].sort_values(by='purchase_count', ascending=False).head()

,product_id,day_of_week,purchase_count,max_purchase_count
15899,422879e10f46682990de24d770e7f83d,Wednesday,93,93
36507,99a4788cb24856965c36a24e339b6058,Monday,92,92
41039,aca2eb7d00ea1a7b8ebd4e68314663af,Thursday,89,89
20123,53b36df67ebb7c41585e8d54d6772e08,Tuesday,76,76
13534,389d119b48cf3043d311335e499d9c6b,Thursday,67,67


#### 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Нужно учесть, что количество недель в месяце не целое.

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Дублирую дф, определённый как покупка
</div>

In [155]:
task4 = task1.copy()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Меняю дату на месяца
</div>

In [156]:
task4['order_purchase_timestamp'] = task4.order_purchase_timestamp.dt.strftime('%Y-%m')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Считаю количество покупок в месяц
</div>

In [157]:
orders_in_month = task4.groupby(['customer_unique_id','order_purchase_timestamp']).size().reset_index(name='order_cnt')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Создаю колонку с количеством дней в месяце
</div>

In [158]:
orders_in_month['days'] = pd.to_datetime(orders_in_month.order_purchase_timestamp)

In [159]:
orders_in_month.days = orders_in_month.days.dt.days_in_month

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Считаю количество недель в месяце
</div>

In [160]:
orders_in_month['weeks'] = orders_in_month.days / 7

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Считаю количество покупок в неделю по месяцам
</div>

In [161]:
orders_in_month['purcheses_a_week'] = orders_in_month.order_cnt/orders_in_month.weeks


In [162]:
orders_in_month

,customer_unique_id,order_purchase_timestamp,order_cnt,days,weeks,purcheses_a_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,31,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,31,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,31,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,31,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,30,4.285714,0.233333
...,...,...,...,...,...,...
97911,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,2,30,4.285714,0.466667
97912,fffea47cd6d3cc0a88bd621562a9d061,2017-12,1,31,4.428571,0.225806
97913,ffff371b4d645b6ecea244b27531430a,2017-02,1,28,4.000000,0.250000
97914,ffff5962728ec6157033ef9805bacc48,2018-05,1,31,4.428571,0.225806


#### 5. Выполнить когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Дублирую дф, определённый как покупка
</div>

In [165]:
task5 = task1.copy()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Добавляю дату первой покупки в новый столбец
</div>

In [166]:
task5['first_purchase_month'] = task5.groupby('customer_unique_id')['order_purchase_timestamp'].transform('min').dt.to_period('M')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Создаю колонку с месяцем покупки
</div>

In [168]:
task5['month'] = task5['order_purchase_timestamp'].dt.to_period('M')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Выявляю количество пользователей, которые совершили первую покупку в определённый месяц и были активны в другие
</div>

In [169]:
cohort_data = task5.groupby(['first_purchase_month', 'month']).agg({'customer_unique_id': 'nunique'}).reset_index()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Создаю столбец с когортным периодом</div>

In [173]:
cohort_data['cohort_period'] = (cohort_data['month'] - cohort_data['first_purchase_month']).apply(lambda x: x.n)

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Делаю сводную таблицу</div>

In [175]:
cohort_pivot = cohort_data.pivot_table(index='first_purchase_month', columns='cohort_period', values='customer_unique_id')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Беру размер когорт (первый столбец)</div>

In [179]:
cohort_size = cohort_pivot.iloc[:, 0]

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Расчитываю ретеншн - процент клиентов, оставшихся активными в каждом когортном периоде, по сравнению с размером когорты:</div>

In [180]:
retention = cohort_pivot.divide(cohort_size, axis=0)

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Оформляю таблицу</div>

In [183]:
ur_style = (retention
            .style
            .set_caption('User retention by cohort')  # добавляю подпись
            .background_gradient(cmap='viridis')  # раскрашиваю ячейки по столбцам
            .highlight_null('white')  # делаю белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматирую как проценты, NaN заменяем на пустоту
ur_style

cohort_period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
first_purchase_month,,,,,,,,,,,,,,,,,,,,
2016-09,100.00%,,,,,,,,,,,,,,,,,,,
2016-10,100.00%,,,,,,0.32%,,,0.32%,,0.32%,,0.32%,,0.32%,,0.32%,0.63%,0.63%
2016-12,100.00%,100.00%,,,,,,,,,,,,,,,,,,
2017-01,100.00%,0.39%,0.26%,0.13%,0.39%,0.13%,0.52%,0.13%,0.13%,,0.39%,0.13%,0.79%,0.39%,0.13%,0.13%,0.26%,0.39%,0.13%,
2017-02,100.00%,0.23%,0.29%,0.12%,0.40%,0.12%,0.23%,0.17%,0.12%,0.23%,0.12%,0.29%,0.17%,0.17%,0.12%,0.06%,0.06%,0.23%,,
2017-03,100.00%,0.49%,0.34%,0.38%,0.34%,0.15%,0.15%,0.30%,0.34%,0.08%,0.38%,0.15%,0.23%,0.11%,0.15%,0.23%,0.08%,0.15%,,
2017-04,100.00%,0.60%,0.21%,0.17%,0.34%,0.26%,0.34%,0.30%,0.30%,0.17%,0.26%,0.09%,0.09%,0.04%,0.09%,0.09%,0.21%,,,
2017-05,100.00%,0.47%,0.50%,0.39%,0.31%,0.33%,0.42%,0.17%,0.25%,0.31%,0.25%,0.33%,0.25%,0.03%,0.19%,0.25%,,,,
2017-06,100.00%,0.48%,0.35%,0.41%,0.25%,0.38%,0.38%,0.22%,0.13%,0.22%,0.32%,0.35%,0.16%,0.13%,0.19%,,,,,


In [186]:
retention[3].idxmax()

Period('2017-06', 'M')

#### 6. Построить RFM сегментацию пользователей. Для каждого сегмента обозначить границы метрик.

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Дублирую дф, определённый как покупка
</div>

In [187]:
task6 = task1.copy()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Смотрю дату последней покупки
</div>

In [196]:
task6.order_purchase_timestamp.max()

Timestamp('2018-09-03 09:06:57')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Устанвливаю дату от которой буду считать recency
</div>

In [191]:
today = pd.to_datetime('2018-09-04')

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Группируем данные по customer_unique_id и рассчитываем RFM:<br>  
В recency идёт разница даты today и последней покупки<br>  
В frequency подсчитываем количество заказов пользователя<br>  
В monetary считаем общую сумму заказов  
</div>

In [193]:
rfm = task6.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (today - x.max()).days, 'order_id': 'nunique', 'price': 'sum'
    }).rename(columns={'order_purchase_timestamp': 'Recency', 'order_id': 'Frequency', 'price': 'Monetary'}).reset_index()

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Присваиваю оценки RFM метрикам по квантилям.<br>
Из за неравномерного распределения Frequency оценки получились 1 2 3 и 5.<br>
И я применил pd.cut к Frequency потому, что квантиль выдавал мне только один диапазон
</div>

In [194]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm['F_Score'] = pd.cut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5]).astype(int)

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Создание столбца с общим RFM Score
</div>

In [195]:
rfm['RFM_score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Сегментация клиентов на основе RFM Score<br>
<br>
1. Best Customers (RFM_score = 555):<br>
Recency: Эти клиенты совершили покупку недавно, в пределах 0–98 дней (лучший квантиль по "недавности").<br>
Frequency:<br>
Monetary: Эти клиенты тратят наибольшие суммы — от 179,9 до 13,440  (самая высокая группа по тратам).<br>
Описание: Это ваши самые ценные клиенты, которые покупают часто, недавно и тратят большие суммы.<br>
<br>
2. Loyal Customers (Recency начинается с 5):<br>
Recency: Эти клиенты также совершили покупку недавно — от 0 до 98 дней.<br>
Frequency: Эти клиенты покупают на регулярной основе — высокие значения в пределах от 1 до 16 покупок.<br>
Monetary: Могут тратить немного меньше, чем лучшие клиенты — диапазон сумм находится между 69.9 и 179.9<br>
Описание: Это лояльные клиенты, которые покупают регулярно и недавно, но их траты могут быть немного ниже, чем у лучших клиентов.<br>
<br>
3. At Risk (Recency начинается с 1):<br>
Recency: Клиенты, которые давно не совершали покупок — от 389 до 729 дней.<br>
Frequency: Эти клиенты, вероятно, покупали редко — от 1 до 16 покупок.<br>
Monetary: Суммы покупок могли варьироваться от 8.5 до 39.9<br>
Описание: Это клиенты, которые находятся в зоне риска, они не возвращались давно и не были особенно активны.<br>
<br>
4. Others:<br>
Recency: Эти клиенты могут совершать покупки в среднем диапазоне времени — от 98 до 389 дней назад.<br>
Frequency: Частота их покупок, как правило, варьируется от 1 до 16 покупок.<br>
Monetary: Траты могут находиться в диапазоне от 39.9 до 179.9<br>
Описание: Это клиенты со средней активностью и тратами, которые не выделяются ни в лучшую, ни в худшую сторону.<br>
<br>
Примечание:<br>
Границы для Recency варьируются от 0 до 729 дней, что указывает на недавних клиентов и тех, кто долго не возвращался.<br>
Frequency имеет диапазон от 1 до 16 заказов<br>
Monetary варьируется значительно, с минимальной тратой в 8.5 и максимальной до 13,440<br>
</div>

In [197]:
def segment_rfm(df):
    if df['RFM_score'] == '555':
        return 'Best Customers'
    elif df['RFM_score'][0] == '5':
        return 'Loyal Customers'
    elif df['RFM_score'][0] == '1':
        return 'At Risk'
    else:
        return 'Others'
    
rfm['Segment'] = rfm.apply(segment_rfm, axis=1)

In [89]:
rfm

,customer_unique_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_score,Segment
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90,4,1,4,414,Others
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90,4,1,1,411,Others
2,0000f46a3911fa3c0805444483337064,542,1,69.00,1,1,2,112,At Risk
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99,2,1,1,211,Others
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00,2,1,5,215,Others
...,...,...,...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,452,1,1570.00,1,1,5,115,At Risk
95416,fffea47cd6d3cc0a88bd621562a9d061,267,1,64.89,3,1,2,312,Others
95417,ffff371b4d645b6ecea244b27531430a,573,1,89.90,1,1,3,113,At Risk
95418,ffff5962728ec6157033ef9805bacc48,124,1,115.00,4,1,4,414,Others


<div style="background-color: #d9f2fa; padding: 10px; border-radius: 5px;">
Границы квантилей
</div>

In [198]:
recency_quantiles = pd.qcut(rfm['Recency'], 5, labels=False, retbins=True, duplicates='drop')
frequency_quantiles = pd.cut(rfm['Frequency'], 5, labels=False, retbins=True, duplicates='drop')
monetary_quantiles = pd.qcut(rfm['Monetary'], 5, labels=False, retbins=True, duplicates='drop')

print("Границы Recency:\n", recency_quantiles[1])
print("Границы Frequency:\n", frequency_quantiles[1])
print("Границы Monetary:\n", monetary_quantiles[1])

Границы Recency:
 [  0.  98. 183. 274. 389. 729.]
Границы Frequency:
 [ 0.984  4.2    7.4   10.6   13.8   17.   ]
Границы Monetary:
 [    0.     39.     69.    109.9   179.9 13440. ]
